In [1]:
# import standard packages
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import importlib
import sklearn
import scipy

In [2]:
# load dataset
vowel_train = pd.read_csv('https://web.stanford.edu/~hastie/ElemStatLearn/datasets/vowel.train')
vowel_test = pd.read_csv('https://web.stanford.edu/~hastie/ElemStatLearn/datasets/vowel.test')
print(vowel_train.shape)
print(vowel_test.shape)

(528, 12)
(462, 12)


In [17]:
# import sklearn packages
from sklearn import preprocessing

# divide into X and Y
x_train = vowel_train.drop('y', axis=1)
y_train = vowel_train['y']
x_test = vowel_test.drop('y', axis=1)
y_test = vowel_test['y']
# convert into numpy array
x_train, x_test = np.array(X_train), np.array(X_test)
y_train, y_test = np.array(y_train), np.array(y_test)
# check results
print("X_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)
n, d = X_train.shape

X_train shape: (528, 11)
y_train shape: (528,)
X_test shape: (462, 11)
y_test shape: (462,)


In [54]:
def get_data(X, y, classes, mult):
    """
    Gets appropriate data for ovo or ovr multiclassification.
    """
    if mult == 'ovo':
        # get y and indicies
        c1, c2 = classes[0], classes[1]
        c1_ind, c2_ind = np.where(y == c1)[0], np.where(y == c2)[0]
        y_sub = y[(y == c1) | (y == c2)]
        # relabel y
        y_sub[y_sub == c1] = -1.0
        y_sub[y_sub == c2] = 1.0
        # get X
        X_sub = np.concatenate([X[c1_ind,:], X[c2_ind,:]])
        return X_sub, y_sub
    elif mult == 'ovr':
        # relabel y
        y_relab = y.copy()
        y_relab[y_relab != classes] = -1.0            
        y_relab[y_relab == classes] = 1.0
        return X, y_relab

x_train, y_train = get_data(x_train, y_train, 1, 'ovr')
x_test, y_test = get_data(x_test, y_test, 1, 'ovr')

In [36]:
def computegrad(beta, lambduh, x=x_train, y=y_train, h=0.5):
    yt = y*x.dot(beta)
    ell_prime = -(1+h-yt)/(2*h)*y*(np.abs(1-yt) <= h) - y*(yt < (1-h))
    return np.mean(ell_prime[:, np.newaxis]*x, axis=0) + 2*lambduh*beta

def objective(beta, lambduh, x=x_train, y=y_train, h=0.5):
    yt = y*x.dot(beta)
    ell = (1+h-yt)**2/(4*h)*(np.abs(1-yt) <= h) + (1-yt)*(yt < (1-h))
    return np.mean(ell) + lambduh*np.dot(beta, beta)

def bt_line_search(beta, lambduh, eta=1, alpha=0.5, betaparam=0.8, maxiter=1000, x=x_train, y=y_train):
    grad_beta = computegrad(beta, lambduh, x=x, y=y)
    norm_grad_beta = np.linalg.norm(grad_beta)
    found_eta = 0
    iter = 0
    while found_eta == 0 and iter < maxiter:
        if objective(beta - eta * grad_beta, lambduh, x=x, y=y) < \
            objective(beta, lambduh, x=x, y=y) \
                - alpha * eta * norm_grad_beta ** 2:
            found_eta = 1
        elif iter == maxiter - 1:
            print('Warning: Max number of iterations of' + 'backtracking line search reached')
            break
        else:
            eta *= betaparam
            iter += 1
    return eta

def mylinearsvm(beta_init, theta_init, lambduh, eta_init, maxiter, x=x_train, y=y_train, eps=1e-6):
    beta = beta_init
    theta = theta_init
    grad_theta = computegrad(theta, lambduh, x=x, y=y)
    beta_vals = beta
    theta_vals = theta
    iter = 0
    while iter < maxiter and np.linalg.norm(grad_theta) > eps:
        eta = bt_line_search(theta, lambduh, eta=eta_init, x=x, y=y)
        beta_new = theta - eta*grad_theta
        theta = beta_new + iter/(iter+3)*(beta_new-beta)
        # Store all of the places we step to
        beta_vals = np.vstack((beta_vals, beta_new))
        theta_vals = np.vstack((theta_vals, theta))
        grad_theta = computegrad(theta, lambduh, x=x, y=y)
        beta = beta_new
        iter += 1
    return beta_vals, theta_vals

def misclassification_error(beta_dict, scaler_dict, nclasses, x=x_test, y=y_test):
    predictions = np.zeros((len(x), int(nclasses * (nclasses - 1) / 2)))
    k = 0
    for i in range(1, nclasses + 1):
        for j in range(i + 1, nclasses + 1):
            # Standardize
            xnow = scaler_dict[(i,j)].transform(x)
            # Predict
            betas = beta_dict[(i, j)]
            xbeta = xnow.dot(betas)
            predictions[:, k] = (xbeta > 0) * i + (xbeta < 0) * j
            k += 1
    y_pred = np.ravel(scipy.stats.mode(predictions, axis=1)[0])
    assert len(y_pred) == len(y)
    return np.mean(y_pred != y), predictions

def objective_plot(betas_fg, lambduh, x=x_train, y=y_train, save_file=''):
    num_points = np.size(betas_fg, 0)
    objs_fg = np.zeros(num_points)
    for i in range(0, num_points):
        objs_fg[i] = objective(betas_fg[i, :], lambduh, x=x, y=y)
    fig, ax = plt.subplots()
    ax.plot(range(1, num_points + 1), objs_fg)
    plt.xlabel('Iteration')
    plt.ylabel('Objective value')
    plt.title('Objective value vs. iteration when lambda=' + str(lambduh))
    if not save_file:
        plt.show()
    else:
        plt.savefig(save_file)

def create_train_set(x_train, y_train, i, j):
    i_idxs_train = np.where(y_train == i)[0]
    xi_train = x_train[i_idxs_train, :]
    yi_train = np.ones_like(i_idxs_train)
    j_idxs_train = np.where(y_train == j)[0]
    xj_train = x_train[j_idxs_train, :]
    yj_train = np.ones_like(j_idxs_train) * -1
    x_train_c = np.vstack((xi_train, xj_train))
    y_train_c = np.concatenate((yi_train, yj_train))
    return x_train_c, y_train_c

def train(lambduh, x_train, y_train, x_test, y_test):
    betas_dict = {}
    scaler_dict = {}
    nclasses = int(np.max(y_train) - np.min(y_train) + 1)
    for i in range(1, nclasses + 1):
        for j in range(i + 1, nclasses + 1):
            x_train_c, y_train_c = create_train_set(x_train, y_train, i, j)
            # Standardize the data.
            scaler = sklearn.preprocessing.StandardScaler()
            x_train_c = scaler.fit_transform(x_train_c)
            scaler_dict[(i,j)] = scaler
            # Initialize
            beta_init = np.zeros(d)
            theta_init = np.zeros(d)
            eta_init = 1 / (scipy.linalg.eigh(1 / len(y_train_c) * \
            x_train_c.T.dot(x_train_c), eigvals=(d - 1, d - 1), eigvals_only=True)[0] + lambduh)
            maxiter = 100
            # Fit the model
            betas_fastgrad, _ = mylinearsvm(beta_init, theta_init,
            lambduh, eta_init, maxiter, x=x_train_c, y=y_train_c)
            betas_dict[(i, j)] = betas_fastgrad[-1, :]
    test_error, preds = misclassification_error(betas_dict, scaler_dict, nclasses, x_test, y_test)
    return test_error, preds

In [55]:
lambduh = 1.0
test_error, preds_corr = train(lambduh, x_train, y_train, x_test, y_test)
print('Test misclassification error when lambda=', lambduh, ':', test_error)

IndexError: too many indices for array

In [53]:
i = 50
print(preds_corr[i,:])
print(scipy.stats.mode(preds_corr[i,:])[0][0])

[ 2.  3.  4.  1.  6.  7.  1.  1.  1. 11.  3.  2.  2.  2.  2.  2.  2.  2.
  2.  3.  3.  3.  3.  3.  3.  3.  3.  4.  4.  4.  4.  4.  4. 11.  6.  7.
  5.  9.  5. 11.  6.  6.  6.  6. 11.  7.  7.  7. 11.  9.  8. 11.  9. 11.
 11.]
3.0


In [26]:
def cross_validate(X, y, lambda_max=2 ** 3, max_iter=1000, num_lambda=10, num_folds=5, seed=0):
    # Run cross-validation to find the optimal lambda
    # Randomly divide the data into num_folds parts
    np.random.seed(seed)
    n = len(y)
    order = list(range(n))
    np.random.shuffle(order)
    idxs = [order[int(i * n / num_folds):int((i + 1) * n / num_folds)] for i in range(num_folds)]
    lambduh = lambda_max
    beta = np.zeros(np.size(X, 1))
    best_error = np.inf
    print('lambda \t Misclassification error')
    # Try many possible lambdas and see which gives the lowest
    # average mse on the test set
    for j in range(num_lambda):
        avg_error = 0
        for part_num in range(num_folds):
            # Create the training and test sets
            x_test = X[idxs[part_num], :]
            y_test = y[idxs[part_num]]
            train_idxs = idxs[0:part_num] + idxs[part_num + 1:]
            train_idxs = [item for sublist in train_idxs for item in sublist]
            x_train = X[train_idxs, :]
            y_train = y[train_idxs]
            # Compute the optimal betas. Use warm-start.
            error = train(lambduh, x_train, y_train, x_test, y_test)
            avg_error += error
        avg_error /= num_folds
        print('%0.2e' % lambduh, '\t', np.round(avg_error, 5))
        # Update the best error
        if avg_error < best_error:
            best_error = avg_error
            best_lambda = lambduh
        lambduh /= 2
    return best_lambda, best_error

In [27]:
best_lambda, error = cross_validate(x_train, y_train, num_lambda=25)
print('The best lambda found was lambda = %0.2e' % best_lambda)
test_error = train(best_lambda, x_train, y_train, x_test, y_test)
print('Test misclassification error when lambda=', best_lambda, ':', test_error)

lambda 	 Misclassification error
8.00e+00 	 0.41279
4.00e+00 	 0.41279
2.00e+00 	 0.40331
1.00e+00 	 0.35786
5.00e-01 	 0.28591
2.50e-01 	 0.24047
1.25e-01 	 0.21588
6.25e-02 	 0.20836
3.12e-02 	 0.18753
1.56e-02 	 0.16293
7.81e-03 	 0.15344
3.91e-03 	 0.14593
1.95e-03 	 0.14397
9.77e-04 	 0.14774
4.88e-04 	 0.14963
2.44e-04 	 0.15152
1.22e-04 	 0.14961
6.10e-05 	 0.1496
3.05e-05 	 0.1496
1.53e-05 	 0.1496
7.63e-06 	 0.1496
3.81e-06 	 0.1496
1.91e-06 	 0.1496
9.54e-07 	 0.1496
4.77e-07 	 0.1496
The best lambda found was lambda = 1.95e-03
Test misclassification error when lambda= 0.001953125 : 0.525974025974026
